In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA

In [ ]:
# Carregar e preparar dados
data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target
df['diagnosis'] = df['target'].map({0: 'Maligno', 1: 'Benigno'})

# Features para clustering
features_clustering = ['mean radius', 'mean texture', 'mean perimeter', 'mean area']
X = df[features_clustering]

# Normalizar dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Encontrar número ideal de clusters
inertia = []
silhouette_scores = []
K_range = range(2, 6)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_scaled, kmeans.labels_))

# Plotar resultados
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

ax1.plot(K_range, inertia, 'bo-')
ax1.set_xlabel('Número de Clusters (K)')
ax1.set_ylabel('Inércia')
ax1.set_title('📉 Método do Cotovelo')

ax2.plot(K_range, silhouette_scores, 'ro-')
ax2.set_xlabel('Número de Clusters (K)')
ax2.set_ylabel('Score de Silhueta')
ax2.set_title('📊 Score de Silhueta')

plt.tight_layout()
plt.show()

In [ ]:
# Aplicar K-means com K=3 (baseado na análise)
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(X_scaled)
df['cluster'] = clusters

print("📊 Distribuição dos Clusters:")
print(df['cluster'].value_counts().sort_index())

In [ ]:
# Visualizar clusters
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
scatter = plt.scatter(df['mean radius'], df['mean texture'], 
                     c=df['cluster'], cmap='viridis', alpha=0.7)
plt.xlabel('Raio Médio')
plt.ylabel('Textura Média')
plt.title('🎯 Clusters - Raio vs Textura')
plt.colorbar(scatter)

plt.subplot(1, 3, 2)
scatter = plt.scatter(df['mean perimeter'], df['mean area'], 
                     c=df['cluster'], cmap='viridis', alpha=0.7)
plt.xlabel('Perímetro Médio')
plt.ylabel('Área Média')
plt.title('📐 Clusters - Perímetro vs Área')
plt.colorbar(scatter)

# PCA para visualização 2D
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

plt.subplot(1, 3, 3)
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=df['cluster'], cmap='viridis', alpha=0.7)
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.title('🔮 Visualização PCA dos Clusters')
plt.colorbar(scatter)

plt.tight_layout()
plt.show()

In [ ]:
# Análise dos clusters
cluster_analysis = df.groupby('cluster').agg({
    'mean radius': 'mean',
    'mean texture': 'mean', 
    'mean perimeter': 'mean',
    'mean area': 'mean',
    'diagnosis': lambda x: x.value_counts().to_dict()
}).round(2)

print("📈 Características Médias por Cluster:")
print(cluster_analysis)

In [ ]:
# Cruzamento clusters vs diagnóstico real
cross_tab = pd.crosstab(df['cluster'], df['diagnosis'])
print("\n🎯 Cruzamento: Clusters vs Diagnóstico Real:")
print(cross_tab)

# Plotar relação clusters vs diagnóstico
plt.figure(figsize=(10, 6))
cross_tab.plot(kind='bar', stacked=True, color=['red', 'green'])
plt.title('🔍 Cluster vs Diagnóstico Real')
plt.xlabel('Cluster')
plt.ylabel('Quantidade')
plt.legend(title='Diagnóstico')
plt.xticks(rotation=0)
plt.show()

In [ ]:
# Interpretação dos clusters
print("\n💡 INTERPRETAÇÃO DOS CLUSTERS:")
print("Cluster 0: Tumores Pequenos/Medios (Provavelmente Benignos)")
print("Cluster 1: Tumores Grandes (Provavelmente Malignos)") 
print("Cluster 2: Tumores Intermediários (Casos Limítrofes)")

print(f"\n🎉 DESCOBERTA: Identificamos um grupo intermediário (Cluster 2) com {len(df[df['cluster']==2])} casos")
print("   que podem requerer atenção especial!")
